In [ ]:
import sys
from chart_studio import plotly
import plotly.graph_objs as go
import plotly.offline as pltlyoff
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Custom files ###
dir_GPPlotly = 'C:\\Users\\tln229\\Downloads\\Python Tests\\Constrained GP_2\\GP_2\\gp_plotly\\'
sys.path.append(dir_GPPlotly)

dir_gp_constr = 'C:\\Users\\tln229\\Downloads\\Python Tests\\Constrained GP_2\\GP_2\\gp_constr\\'
sys.path.append(dir_gp_constr)

# Import
from GPPlotly.plottingfunctions import PlotGP2d, add_traces_to_fig, gp_diagnostics, pred_vs_error_perc
from GPConstr.model import GPmodel, Constraint
from GPConstr.kern import kernel_Matern52

In [2]:
def get_diagnostics_df(x_test, y_test, mean, var):
    """ Put test data in a dataframe"""
    df = pd.DataFrame(x_test)
    df.columns = ['x' + str(i+1) for i in range(x_test.shape[1])]
    df['y_true'] = y_test
    df['y_mean'] = mean
    df['y_var'] = var
    
    return df

In [3]:
# ---------- DATA ----------
def func(x):
    return x[:, 0] - x[:, 1]

# training
n_train = 20
x1 = np.linspace(0, 10, n_train).reshape(-1, 1)
x2 = np.random.uniform(0, 10, n_train).reshape(-1, 1)

x_train = np.concatenate((x1, x2), axis = 1)
y_train = func(x_train) + np.random.uniform(-2, 2, n_train)

# Xv
x1v = np.linspace(0, 10, 10).reshape(-1, 1)
x2v = np.random.uniform(0, 10, 10).reshape(-1, 1)
Xv  = np.concatenate((x1, x2), axis = 1)

# testing
n_test = 10
x1_test = np.linspace(0, 10, n_test).reshape(-1, 1)
x2_test = np.random.uniform(0, 10, n_test).reshape(-1, 1)

x_test = np.concatenate((x1_test, x2_test), axis = 1)
y_test = func(x_test)

In [4]:
# ----- MODEL -----
ker = kernel_Matern52(variance = 1, lengthscale = [1]*x_train.shape[1])
model = GPmodel(kernel = ker, likelihood = 1, mean = 0)

# feed data to model
model.X_training = x_train
model.Y_training = y_train

In [5]:
# ----- OPTIMIZE -----
model.optimize(include_constraint = False, fix_likelihood = False)

..Running optimization for unconstrained GP ... DONE - Total time: 0.035 seconds


In [6]:
# ----- CONSTRAINTS -----
def constant_function(val):
    def fun(x):
        return np.array([val]*x.shape[0])
    return fun

model.constr_bounded    = Constraint(LB = constant_function(-10), UB = constant_function(10))
model.constr_bounded.Xv = Xv

model.constr_likelihood = 1E-6

In [7]:
# Predict
mean, var, perc, mode, samples, sampling_time = model.calc_posterior_constrained(x_test, num_samples = 1000, save_samples = 5)
mean = np.array(mean).flatten()
lower = perc[0]
upper = perc[2]
var = np.array(var).flatten()

# Plot
fig = pred_vs_error_perc(mean, lower, upper, y_test, 95)
pltlyoff.iplot(fig, filename='')

..Running calculation of K_w ... SKIP - (cached)
..Running calculation of Cholesky factor for K_w ... SKIP - (cached)
..Running preparation step 1 - dependence on (XS, X) ... DONE - time: 0.001 seconds
..Running preparation step 2 - dependence on (XV, X) ... DONE - time: 0.001 seconds
..Running preparation step 3 - dependence on (XS, XV, X) ... DONE - time: 0.001 seconds
..sampling 1000 times from truncated constraint distribution C~|C, Y DONE - time: 0.009 seconds
..sampling 1000 times from constrained GP f*|C, Y DONE - time: 0.001 seconds
..computing statistics from samples DONE - time: 0.001 seconds
 DONE - Total time: 0.014 seconds
